In [46]:
import random, sys, time, math, pygame
from pygame.locals import *
FPS = 30
WINWIDTH = 640
WINHEIGHT = 480
HALF_WINWIDTH = int(WINWIDTH / 2)
HALF_WINHEIGHT = int(WINHEIGHT / 2)

GRASSCOLOR = (24, 255, 0)
WHITE = (255, 255, 255)
RED = (255,0 ,0)

CAMERASLACK = 90
MOVERATE = 9
BOUNCERATE = 6
STARTSIZE = 25
WINSIZE = 300
INVULNTIME = 2
GAMEOVERTIME = 4
MAXHEALTH = 3
NUMGRASS = 80
NUMSQUIRRLES = 30
SQUIRRLEMINSPEED = 3
SQUIRRLEMAXSPEED = 7
DIRCHANGEFREQ = 2
LEFT = "left"
RIGHT = "right"

def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT, L_SQUIR_IMG, R_SQUIR_IMG, GRASSIMAGES
    
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    pygame.display.set_icon(pygame.image.load('gameicon.png'))
    DISPLAYSURF = pygame.display.set_mode((WINWIDTH, WINHEIGHT))
    pygame.display.set_caption("Squirrel Eat Squirrel")
    BASICFONT = pygame.font.Font("freesansbold.ttf", 32)
    
    L_SQUIR_IMG = pygame.image.load("squirrel.png")
    R_SQUIR_IMG = pygame.transform.flip(L_SQUIR_IMG, True, False)
    GRASSIMAGES = []
    for i in range (1, 5):
        GRASSIMAGES.append(pygame.image.load("grass%s.png" % i))
    while (True):
        runGame()
def runGame():
    invulnerbleMode = False
    invulnerbleStartTime = 0
    gameOverMode = False
    gameOverStartTime = 0
    winMode = False
    gameOverSurf = BASICFONT.render("game over", True, WHITE)
    gameOverRect = gameOverSurf.get_rect()
    gameOverRect.center = (HALF_WINWIDTH, HALF_WINHEIGHT)
    
    winSurf = BASICFONT.render("you have achieved OMEGA SQUIRREL", True, WHITE)
    winRect = winSurf.get_rect()
    winRect.center = (HALF_WINWIDTH, HALF_WINHEIGHT)
    
    winSurf2 = BASICFONT.render("Press r to Restart", True, WHITE)
    winRect2 = winSurf2.get_rect()
    winRect2.center = (HALF_WINWIDTH, HALF_WINHEIGHT + 30)
    camerax = 0
    cameray = 0
    grassObjs = []
    squirrelObjs = []
    playerObjs = {"surface": pygame.transform.scale(L_SQUIR_IMG, (STARTSIZE, STARTSIZE)), "facing": LEFT, "size": STARTSIZE, "x": HALF_WINWIDTH, "y": HALF_WINHEIGHT, "bounce": 0, "health": MAXHEALTH}
    
    moveLeft = False
    moveRight = False
    moveUp    = False
    moveDown = False
    
    for i in range (10):
        grassObjs.append(makeNewGrass(camerax, cameray))
        grassObjs[i]["x"] = random.randint(0, WINWIDTH)
        grassObjs[i]["y"] =random.randint(0, WINHEIGHT)
    while True:
        if invulnerableMode and time.time() - invulnerableStartTime > INVULNTIM:
            invulnerableMode = False
        for sObj in squirrelObjs:
            sObj["x"] += sObj[movex]
            sObj["y"] += sObj[movey]
            sObj["bounce"] += 1
            if sObj["bounce"] > sObj["bouncerate"]:
                sObj["bounce"] = 0
            
            if random.randint(0, 99) < DIRCHANGEFREQ:
                sObj["movex"] = getRandomVelocity()
                sObj["movey"] = getRandomVelocity()
                if sObj["movex"] > 0:
                    sObj["surface"] = pygame.transform.scale(R_SQUIR_IMG, (sObj["width"],sObj["height"]))
                else:
                    sObj["surface"] = pygame.transform.scale(L_SQUIR_IMG, (sObj["width"],sObj["height"]))
                for i in range(len(grassObjs) -1, -1, -1):
                    if isOutsideActiveArea(camerax, cameray, grassObjs[i]):
                        del grassObjs[i]
                for i in range (len(squirrelObjs) -1, -1, -1):
                    if isOutsideActiveArea(camerax, cameray, squirrelObjs[i]):
                        del squirrelObjs[i]
                
                while len(grassObjs) < NUMGRASS:
                    grassObjs.append(makeNewGrass(camerax, cameray))
                while len(squirrelObjs) < NUMSQUIRRELS:
                    squirrekObjs.append(makeNewSquirrel(camerax, cameray))
                    
                playerCenterx = playerObj["x"] + int(playerObj["size"] / 2)
                playerCentery = playerObj["y"] + int(playerObj["size"] / 2)
                if (camerax + camerax) - playerCenterx > CAMERASLACK:
                    camerax = playerCenterx + CAMERASLACK - HALF_WINWIDTH
                elif playerCenterx - (camerax + HALF_WINWIDTH) > CAMERASLACK:
                    camerax = playerCenterx - CAMERASLACK - HALF_WINWIDTH
                if (cameray + HALF_WINHEIGHT) - playerCentery > CAMERASLACK:
                    cameray = playerCentery + CAMERASLACK - HALF_WINHEIGHT
                elif playerCentery - (cameray + HALF_WINHEIGHT) > CAMERASLACK:
                    cameray = playerCentery - CAMERASLACK - HALF_WINHEIGHT
                
                DISPLAYSURF.fill(GRASSCOLOR)
                
                for gObj in grassObjs:
                    gRect = pygame.Rect((gObj["x"] - camerax, gObj["y"] - cameray, gObj["width"], gObj["height"]))
                    DISPLAYSURF.blit(GRASSIMAGES[gObj["grassImage"]], gRect)
                
                for sObj in squirrelObjs:
                        sObjs["rect"] = pygame.Rect((sObj["x"] - camerax, sObj["y"] - cameray - getBounceAmount(sObj["bounce"], sObj["bouncerate"], sObj["bounceheight"]), sObj["width"], sObj["height"]))
                        DISPLAYSURF.blit(sObj["surface"], sObj["rect"])
                        
                        flashIsOn = round(time.time(), 1) * 10 % 2 == 1
                        if not gameOverMode and not (invulnerableMode and flashIsOn):
                            playerObj["rect"] = pygame.Rect((playerObj["x"] - camerax, playerObj["y"] - cameray - getBounceAmount(playerObj["bounce"], BOUNCERATE, BOUNCEHEIGHT), playerObj["size"], playerObjp["size"]))
                            DISPLAYSURF.blit(playerObj["surface"], playerObj["rect"])
                            
                        drawHealthMeter(playerObj["health"])
                        
                        for event in pygame.event.get():
                            if event.type == QUIT:
                                terminate()
                            elif event.type == KEYDOWN:
                                if event.key in (K_UP, K_w):
                                    moveDown = False
                                    moveUp = True
                                elif event.key in (K_DOWN, K_s):
                                    moveUp = False
                                    moveDown = True
                                elif event.key in (K_LEFT, K_a):
                                    moveRight = False
                                    moveLeft = True
                                    if playerObj["facing"] == RIGHT:
                                        playerObj["surface"] = pygame.transform.scale(L_SQUIR_IMG, (playerObj["size"], playerObj["size"]))
                                        playerObj["facing"] = LEFT
                                elif event.key in (K_RIGHT, K_d):
                                    moveLeft = False
                                    moveRight = True
                                    if playerObj["facing"] == LEFT:
                                        playerObj["surface"] = pygame.transform.scale(R_SQUIR_IMG, (playerObj["size"], playerObj["size"]))
                                        playerObj["facing"] == RIGHT
                                elif winMode and event.key == K_r:
                                    return
                                elif event.type == KEYUP:
                                    if event.key in (K_LEFT, K_a):
                                        moveLeft = False
                                    elif event.key in (K_RIGHT, K_d):
                                        moveRight = False
                                    elif event.key in (K_UP, K_w):
                                        moveUp = False
                                    elif event.key in (K_DOWN, K_s):
                                        moveDown = False
                                    
                                    elif event.key == K_ESCAPE:
                                        terminate()
                                    
                                    if not gameOberMode:
                                        if moveLeft:
                                            playerObj["x"] -= MOVERATE
                                        if moveRight:
                                            playerObj["x"] += MOVERATE
                                        if moveUp:
                                            playerObj["y"] -= MOVERATE
                                        if moveDown:
                                            playerObj["y"] += MOVERATE
                                    if (moveLeft or moveRight or moveUp or moveDown or playerObj["bounce"]) !=0:
                                        playerObj["bounce"] += 1
                                    if playerObj > BOUNCERATE:
                                        playerObj["bounce"] = 0
                                    
                                    for i in range(len(squirrelObjs) -1, -1, -1):
                                        sqObj = squirrelObjs[i]
                                        if "rect" in sqObj and playerObj["rect"].colliderect(sqObj["rect"]):
                                            if sqObj["width"] * sqObj["height"] <= playerObj["size"] **2:
                                                playerObj["size"] += int((sqObj["width"] * sqObj["height"])**0.2) + 1
                                                del squirrelOjs[i]
                                                if playerObj["facing"] == LEFT:
                                                    playerObj["surface"] = pygame.transform.scale(L_SQUIR_IMG, (playerObj["size"], playerObj["size"]))
                                                if playerObj["facing"] == RIGHT:
                                                    playerObj["surface"] = pygame.transform.scale(R_SQUIR_IMG, (playerObj["size"], playerObj["size"]))
                                                    
                                                if playerObj["size"] > WINSIZE:
                                                    winMode = True
                                        elif not invulnerableMode:
                                            invulnerableMode = True
                                            invulnerableStartTIme = time.time()
                                            playerObj["health"] -= 1
                                            if playerObj["health"] == 0:
                                                gameOverMode = True
                                                gameOverStartTime = time.time()
                                    else:
                                        DISPLAYSURF.blit(gameOverSurf, gameOverRect)
                                        if time.time() - gameOverStartTime > GAMEOVERTIME:
                                            return
                                    if winMode:
                                        DISPLAYSURF.blit(winSurf, winRect)
                                        DISPLAYSURF.blit(winSurf2, winRect2)
                                    pygame.display.update()
                                    FPSCLOCK.tick(FPS)
def drawHealthMeter(currentHealth):
    for i in range(currentHealth):
        pygame.draw.rect(DISPLAYSURF, RED, (15, 5 + ( 10 * MAXHEALTH) - i * 10, 20, 10))
    for i in range(MAXHEALTH):
        pygame.draw.rect(DISPLAYSURF, WHITE, (15, 5 + ( 10 * MAXHEALTH) - i * 10, 20, 10), 1)
def terminate():
    pygame.quit()
    sys.exit()
def getBounceAmount(currentBounce, bounceRate, bounceHeight):
    return int(math.sin((math.pi / float(bounceRate)) * currentBounce) * bounceHeight)
def getRandomVelocity():
    speed = random.randint(SQUIRRELMINSPEED, SQUIRRELMAXSPEED)
    if random.randint(0, 1) == 0:
        return speed
    else:
        return -speed
def getRandomOffCameraPos(camerax, cameray, objWidth, objHeight):
    cameraRect = pygame.rect(camerax, cameray, WINWIDTH, WINHEIGHT)
    while True:
        x = random.randint(camerax - WINWIDTH, camerax + (2 * WINWIDTH))
        y = random.randint(camerax - WINHEIGHT, camerax + (2 * WINHEIGHT))
        objRect = pygame.Rect(x, y, objWidth, objHeight)
        if not objRect.colliderect(cameraRect):
            return x, y
def makeNewSquirrel(camerax, cameray):
    sq = {}
    generalSize = random.randint(5, 25)
    multiplier = random.randint(1, 3)
    sq["width"] = (generalSize + random.randint(0, 10)) * multiplier
    sq["height"] = (generalSize + random.randint(0, 10)) * multiplier
    sq["x"], sq["y"] = getRandomOffCameraPos(camerax, cameray, sq["width"], sq["height"])
    sq["movex"] = getRandomVelocity()
    sq["movey"] = getRandomVelocity()
    if sq["movex"] < 0:
        sq["surface"] = pygame.transform.scale(L_SQUIR_IMG, (sq["width"],sq["height"]))
    else:
        sq["surface"] = pygame.transform.scale(R_SQUIR_IMG, (sq["width"],sq["height"]))
    sq["bounce"] = 0
    sq["bouncerate"] = random.randint(10, 18)
    sq["bounceheight"] = random.randint(10, 50)
    return sq

def makeNewGrass(camerax, cameray):
    gr = {}
    gr["grassImage"] = random.randint(0, len(GRASSIMAGES) - 1)
    gr["width"] = GRASSIMAGES[0].get_width()
    gr["height"] = GRASSIMAGES[0].get_height()
    gr["x"], gr["y"] = getRandomOffCameraPos(camerax, cameray, gr["width"], gr["height"])
    gr["rect"] = pygame.Rect((gr["x"], gr["y"], tgr["width"], gr["height"]))
    return gr
def isOutsideActiveArea(camerax, cameray, obj):
    boundsLeftEdge = camerax - WINWIDTH
    boundsTopEdge = cameray - WINHEIGHT
    boundsRect = pygame.Rect(obj["x"], obj["y"], obj["width"], obj["height"])
    return not boundsRect.colliderect(objRect)
if __name__ == '__main__':

    main()